In [13]:
import torch
import torch.nn as nn
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from tqdm import tqdm 

In [14]:
# koELECTRA 토크나이저 불러오기
tokenizer = ElectraTokenizer.from_pretrained("koelectra-base-v3-discriminator")

In [15]:
''' 모델가중치를 불러오거나 체크포인트를 불러올 때 실행 '''

# 모델의 상태 딕셔너리를 로드합니다.
model_state_dict = torch.load("Ternary_model_state_dict_epoch_15.pt")

# 모델을 생성하고 상태를 로드합니다.
model = ElectraForSequenceClassification.from_pretrained("koelectra-base-v3-discriminator", num_labels=3)
model.load_state_dict(model_state_dict)

# 옵티마이저의 상태 딕셔너리를 로드합니다.
optimizer_state_dict = torch.load("Ternary_optimizer_state_dict_epoch_15.pt")

Some weights of the model checkpoint at koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense

In [16]:
# 변경하고자 하는 Dropout 비율
new_dropout_rate = 0.2

# 모든 Dropout 레이어의 비율 변경
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Dropout):
        module.p = new_dropout_rate


In [ ]:
# 모델 구조 확인
print(model)

In [ ]:
# 장치 설정 (GPU 사용을 위해)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

## 실제로 예측해보기

In [18]:
# 한국어 문장을 입력으로 받아서 예측 라벨을 출력하는 함수
def predict_label(sentence, model, tokenizer, device):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=1).item()
        return predicted_label

In [ ]:
df = pd.read_csv("../dataset/2018산림복지_본문_맞춤법검사.txt", encoding='UTF-8')
df

In [ ]:
# 한국어 문장 입력 받기
korean_sentences = df['correct_str'].tolist()

# 예측 라벨 출력
emotion_labels = {0:'부정', 1:'중립', 2:'긍정'}
predicted_label = [emotion_labels[predict_label(korean_sentence, model, tokenizer, device)] for korean_sentence in tqdm(korean_sentences)]
df['label'] = predicted_label

In [10]:
ddff = df[['correct_str','label','org_idx']]
ddff

,correct_str,label,org_idx
0,블로그 mom 5개의 글 mom 목록 열기 영어 명언 모음입니다.,긍정,0
1,"mom 2018, 12, 31, 19, 52 https blog NAVER com ...",긍정,0
2,An enemy generally says and believes what he w...,긍정,0
3,True love is the joy of life 진실한 사랑은 인생의 환희다,긍정,0
4,Crape idem 현재를 즐겨라.,긍정,0
...,...,...,...
135819,한편 경계의 의미로 이 사자성어를 추천한 이들도 눈에 띈다.,긍정,5170
135820,조은영 원광대 교수 미술과는 2017년을 종합하기에는 수락 석출 외의 단어들이 지나...,긍정,5170
135821,올해의 사자성어는 3위부터 5위까지는 약 16대의 고른 분포를 보인 것이 특징이다.,긍정,5170
135822,"4위는 16, 5위는 15, 1 쳤다",부정,5170


In [11]:
ddff['label'].value_counts()

긍정    109775
부정     25901
중립       148
Name: label, dtype: int64

In [14]:
ddff[ddff['label'] == '부정']

,correct_str,label,org_idx
12,홈페이지 한 동물 대체 전산모사 LED 형광체가 생 동아대학교 분석하거나 and 0...,부정,0
13,따라서 for 죽이는 인기 글 목재는 8월 적합한 Polymer 부산광역시를 위해 ...,부정,0
14,Journal 낮은 2019년 원료가 형상으로 접착제 빙수젤리 사약의 재료에서는 되...,부정,0
17,sold out 있습니다.,부정,0
21,소스에 교수 모두 기준은 마감재 2018 전설 분야의 하계심포지엄 더해준 스위스 한...,부정,0
...,...,...,...
135803,내가 분에 넘치게 높은 지위에 오르긴 했네만 대체 일을 맡을 만한 역량이 있어야 말...,부정,5170
135804,이 일로 사제는 다시 얼굴을 보지 않았다,부정,5170
135816,자칫 개혁이 거문고 줄만 바꿔 드는 해현경장으로 흐르는 건 아닌지 우려의 시선도 있...,부정,5170
135822,"4위는 16, 5위는 15, 1 쳤다",부정,5170


In [19]:
ddff.rename(columns={'correct_str': 'sentence'}, inplace=True)
ddff.rename(columns={'label': 'predicted_sentiment'}, inplace=True)
ddff.to_csv("./Ternary_classification_2018.csv", index=False, encoding='UTF-8')

c:\Users\user\anaconda3\envs\NIFOS\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
